In [ ]:
#input must be options chain day1 - end

In [1]:
import wandb
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import joblib
import random
import hashlib

2024-06-23 07:09:40.351723: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
CSV_PATH     = './data/OptionsEOD.csv/'
PARQUET_PATH = './data/OptionsEOD.parquet'
PARQUET_STG_PATH = './data/OptionsEOD_STG.parquet'
SCALER_COL  = ['DTE','INTRINSIC_VALUE','TOTAL_VOLUME',	'C_BID',	'C_ASK', 'C_VOLUME',  'P_BID',	'P_ASK', 'P_VOLUME' ]
SCALER_PATH = './data/scaler.gz'
 
UNIQUE_KEYS = ['QUOTE_DATE','SYMBOL','EXPIRE_DATE']

In [ ]:
#example
EOD_CSV = pd.read_csv(CSV_PATH+"qqq/qqq_eod_201201.txt", engine='pyarrow')
EOD_CSV.head()

In [ ]:
EOD_CSV.columns

In [ ]:
#Part I
#TransformData : 
#-each partition from EXPIRE_DATE 
#-csv too parquet
#-col. rename 
def TransformDataI():
    scaler = MinMaxScaler()
    schema = None
    pqwriter = None
    for d in os.listdir(CSV_PATH):
        for f in os.listdir(CSV_PATH+f"{d}/"):
            if f.endswith(".txt"):
                ## load
                print( f"[LOAD] : {CSV_PATH}{d}/{f}        ",end='\r')
                EOD_CSV = pd.read_csv(CSV_PATH+f"{d}/"+f, engine='pyarrow')
                    
                ## rename col.
                for c in EOD_CSV.columns:
                    EOD_CSV = EOD_CSV.rename( columns={ c:c.strip().replace(']','').replace('[','') } )
                
                ## add symbol 
                EOD_CSV['SYMBOL'] = d.upper()
                ## add INTRINSIC_VALUE
                EOD_CSV['INTRINSIC_VALUE'] = EOD_CSV['UNDERLYING_LAST'] - EOD_CSV['STRIKE']
                
                ## fillnafillna
                EOD_CSV['P_VOLUME'] = EOD_CSV['P_VOLUME'].fillna(0)
                EOD_CSV['C_VOLUME'] = EOD_CSV['C_VOLUME'].fillna(0)


                
                # date columns convert to datetime
                for c in ["QUOTE_READTIME","QUOTE_DATE","EXPIRE_DATE"]:
                    EOD_CSV[c] = pd.to_datetime(EOD_CSV[c])
                
                #clean float data
                for c in ['INTRINSIC_VALUE','C_DELTA','C_GAMMA','C_VEGA','C_THETA','C_RHO','C_IV','C_VOLUME','C_LAST','C_BID','C_ASK','STRIKE','P_BID','P_ASK','P_LAST','P_DELTA','P_GAMMA','P_VEGA','P_THETA','P_RHO','P_IV','P_VOLUME','STRIKE_DISTANCE','STRIKE_DISTANCE_PCT']:
                    if EOD_CSV[c].dtype not in ( 'float32','float64'):
                        EOD_CSV[c] = EOD_CSV[c].apply(lambda x: x.strip())
                        EOD_CSV[c] = EOD_CSV[c].replace('', np.nan).fillna(np.nan)
                        EOD_CSV[c] = EOD_CSV[c].astype('float64')
                    if EOD_CSV[c].dtype == 'float32':
                        EOD_CSV[c] = EOD_CSV[c].astype('float64')
                        
                # REMAIN_DAYS(int) =>  use DTE col.
                #partition with QUOTE_DATE
                EOD_CSV['PartitionDate'] = EOD_CSV['QUOTE_DATE'].dt.strftime('%Y-%m')
                EOD_CSV.sort_values(['QUOTE_DATE','EXPIRE_DATE','SYMBOL','STRIKE'],ascending =False ) 

                #scaler(Normalization_
                #scaler.partial_fit(EOD_CSV[SCALER_COL])

                # save
                if os.path.exists(PARQUET_PATH):
                  EOD_CSV.to_parquet(PARQUET_PATH, engine='fastparquet', append=True, partition_cols=['PartitionDate'], index=False )
                else:
                  EOD_CSV.to_parquet(PARQUET_PATH, engine='fastparquet' , partition_cols=['PartitionDate'], index=False  )
                    
    # joblib.dump(scaler, SCALER_PATH )
    # if pqwriter:
    #     pqwriter.close()
    # print( f"[DONE]                                                       ",end='\r')



In [ ]:
##-RunCleanData
#TransformDataI()

In [5]:
#Part II 
#TransformData : 
# - read each partitions 
# - Normalization if not have scaler.gz file
def strikeZero(df,v,num_rm):
    # First, filter based on QUOTE_DATE, SYMBOL, and EXPIRE_DATE
    filtered_arr = df[(df['QUOTE_DATE'] == v['QUOTE_DATE']) &
                     (df['SYMBOL'] == v['SYMBOL']) &
                     (df['EXPIRE_DATE'] == v['EXPIRE_DATE'])]['STRIKE'].values
    
    # print(max_intrinsic_value)
    # print(min_intrinsic_value)
    df.loc[ (df['QUOTE_DATE'] == v['QUOTE_DATE']) 
            & (df['SYMBOL'] == v['SYMBOL']) 
            & (df['EXPIRE_DATE'] == v['EXPIRE_DATE'])
            & (  (df['STRIKE'].isin(filtered_arr[:5])  )
               | (df['STRIKE'].isin(filtered_arr[-5:])   )
              ) 
        , ['DTE', 'INTRINSIC_VALUE', 'C_BID',	'C_ASK', 'C_VOLUME',  'P_BID',	'P_ASK',	'P_VOLUME']       
    ] = 0
    
def hash_str(S):
    return hashlib.md5(S.encode('utf-8')).hexdigest() 
    
def TransformDataII():
    
    keys = None#df[unique_keys].sort_values(by=unique_keys).drop_duplicates()
    max_option_len = 20
    scaler = MinMaxScaler()
    PartitionDate = [ d[-7:] for d in  os.listdir(PARQUET_PATH) if 'PartitionDate' in d]
    options_qoute = {}
    for i,partdate in enumerate(PartitionDate) :  
        df = pd.read_parquet(PARQUET_PATH,engine='pyarrow'
                                     , filters=[('PartitionDate', '=', partdate)]
                                    )
        #add col options_id
        df['OPTIONS_ID'] = None
        df['P_VOLUME'] = df['P_VOLUME'].fillna(0)
        df['C_VOLUME'] = df['C_VOLUME'].fillna(0)

        ####################################################
        keys = df[UNIQUE_KEYS].sort_values(by=UNIQUE_KEYS).drop_duplicates()
        #loop each keys
        for j,v in keys.iterrows():
            
            df_filter=df[ (df['QUOTE_DATE'] == v['QUOTE_DATE']) & (df['SYMBOL'] == v['SYMBOL']) & (df['EXPIRE_DATE'] == v['EXPIRE_DATE']) ]
            qoute = "".join(v[ ['SYMBOL','EXPIRE_DATE'] ].apply(str).values)
            #add qoute
            if qoute not in [*options_qoute.keys()]:
                options_qoute[qoute] = {}
                options_qoute[qoute]['start_price'] = df_filter['UNDERLYING_LAST'].values[0]
                options_qoute[qoute]['strike'] = df_filter[ df_filter['INTRINSIC_VALUE'].abs().isin(df_filter['INTRINSIC_VALUE'].abs().sort_values()[:max_option_len]) ]['STRIKE'].values
                options_qoute[qoute]['exp'] = df_filter['EXPIRE_DATE'].values[0]
                #check diff UNDERLYING_LAST
                if df_filter['UNDERLYING_LAST'].values[0] != round(np.average(df_filter['UNDERLYING_LAST']),4):
                    print('[ERROR] : set UNDERLYING_LAST ',qoute )
            #rm index max : max_option_len
            rm_strike_index = df_filter[ ~df_filter['STRIKE'].isin(options_qoute[qoute]['strike']) ].index
            df = df.drop(rm_strike_index)
            df_filter = df_filter.drop(rm_strike_index)

            # Generate zero strike 
            # Generate a random float between 0.01 and 1
            random_number = random.uniform(0, 1)
            nom_rm_rows = [r  for r in [0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4] if random_number < r]
            strikeZero(df,v,len(nom_rm_rows) )
            #TOTAL_P_VOLUME,TOTAL_C_VOLUME,OPTIONS_ID
            df.loc[ (df['QUOTE_DATE'] == v['QUOTE_DATE']) 
            & (df['SYMBOL'] == v['SYMBOL']) 
            & (df['EXPIRE_DATE'] == v['EXPIRE_DATE']) 
            , ['TOTAL_VOLUME','OPTIONS_ID']
            ] = [
                df_filter['P_VOLUME'].sum() + df_filter['C_VOLUME'].sum(),
                hash_str(f'{i}{j}')
            ]
            
        #clear expire options_qoute
        for qi in list(options_qoute.keys()):
            if options_qoute[qi]['exp'] < df['QUOTE_DATE'].values[0]:
                options_qoute.pop(qi)
    
        ###################################################

        # scaler.partial_fit
        scaler.partial_fit(df[SCALER_COL])
        print(f"[Processing] {partdate} {round(((i+1)/len(PartitionDate))*100,2)}%     ",end='\r')
        
        # save
        if os.path.exists(PARQUET_STG_PATH):
          df.to_parquet(PARQUET_STG_PATH, engine='fastparquet', append=True, partition_cols=['PartitionDate'], index=False )
        else:
          df.to_parquet(PARQUET_STG_PATH, engine='fastparquet' , partition_cols=['PartitionDate'], index=False  )
            
    joblib.dump(scaler, SCALER_PATH )

In [10]:
TransformDataII()

In [ ]:
#PartIII to H5

In [ ]:
import os
from sklearn.preprocessing import MinMaxScaler
import joblib
import pandas as pd
import h5py
import tensorflow as tf
from IPython.display import clear_output,display, HTML
#================== initialization ==================
LATENT_DIM = 128
VAE_LR = 5e-4
PARQUET_PATH = './data/OptionsEOD_STG.parquet'
SCALER_PATH = './data/scaler.gz'
UNIQUE_KEYS = ['QUOTE_DATE','SYMBOL','EXPIRE_DATE']
SCALER_COL  = ['DTE','INTRINSIC_VALUE', 'TOTAL_VOLUME',	'C_BID',	'C_ASK', 'C_VOLUME',  'P_BID',	'P_ASK',	'P_VOLUME' ]
H5_PATH = './data/OptTrainData/'
EPOCHS = 5

In [ ]:
#Example
import random
from IPython.display import clear_output,display, HTML
import numpy as np
#load scaler
scaler = MinMaxScaler()
PartitionDate = [ d[-7:] for d in  os.listdir(PARQUET_PATH) if 'PartitionDate' in d]
random.shuffle(PartitionDate)
scaler = joblib.load(SCALER_PATH)


for i,partdate in enumerate(PartitionDate) :
    df = pd.read_parquet(PARQUET_PATH,engine='pyarrow'
                                 , filters=[('PartitionDate', '=', partdate)]
                                )
    df['P_VOLUME'] = df['P_VOLUME'].fillna(0)
    df['C_VOLUME'] = df['C_VOLUME'].fillna(0)
    DATA  = np.empty((0,) + (20,9) ) 
    for opt_id in np.unique( df[["OPTIONS_ID"]].values):
        df_filter  = df[df["OPTIONS_ID"]==opt_id]
        if len(df_filter) == 20:
            DATA = np.vstack((DATA ,[scaler.transform(df_filter[SCALER_COL])]))
        else:
            #print( len(df_filter) )
            #display(HTML(df_filter[['STRIKE']+SCALER_COL].to_html()))
            pass
            
    # Save the NumPy array to an HDF5 file
    with h5py.File(H5_PATH+f"{partdate}.h5", 'w') as f:
        dset = f.create_dataset(f'{partdate}', data=DATA, chunks=True , compression='gzip')

    print(f"[Processing] {partdate}, {round(((i+1)/len(PartitionDate))*100,2)}%     ",end='\r')

In [ ]:
#ปรับแก้ กรณี rows < 20 and rows > 20
#try save md5

In [8]:
joblib.dump(scaler, SCALER_PATH )

NameError: name 'scaler' is not defined

In [ ]:
#Example data
PartitionDate = [ d[-7:] for d in  os.listdir(PARQUET_STG_PATH) if 'PartitionDate' in d]
options_qoute = {}
for i,partdate in enumerate(PartitionDate[5:6]) :  
    df = pd.read_parquet(PARQUET_PATH,engine='pyarrow'
        , filters=[('PartitionDate', '=', partdate)]
        )
    break

In [ ]:
keys = df[UNIQUE_KEYS].sort_values(by=UNIQUE_KEYS).drop_duplicates()
#loop each keys
for j,v in keys.iterrows():

    df_filter=df[ (df['QUOTE_DATE'] == v['QUOTE_DATE']) 
        & (df['SYMBOL'] == v['SYMBOL']) 
        & (df['EXPIRE_DATE'] == v['EXPIRE_DATE']) 
    ]

    df_filter=df_filter[
        (df_filter['INTRINSIC_VALUE'].max() == df_filter['INTRINSIC_VALUE'])
    |   (df_filter['INTRINSIC_VALUE'].min() == df_filter['INTRINSIC_VALUE'])
    ]
    break

In [ ]:
3============== sampling show 3===========================

In [ ]:
PartitionDate = [ d[-7:] for d in  os.listdir(PARQUET_STG_PATH) if 'PartitionDate' in d]

In [ ]:
import random
from IPython.display import clear_output,display, HTML

for i in range(10):
    ran_partiion=random.choice(PartitionDate)

    df = pd.read_parquet(PARQUET_STG_PATH,engine='pyarrow'
                                 , filters=[('PartitionDate', '=', ran_partiion)]
                                )
    key = random.choice( [*df[UNIQUE_KEYS].sort_values(by=UNIQUE_KEYS).drop_duplicates().values] )

    df_filtered=df[ (df['QUOTE_DATE'] == key[0]) 
        & (df['SYMBOL'] == key[1]) 
        & (df['EXPIRE_DATE'] == key[2]) 
    ]
    break
    display(HTML(df_filter[['STRIKE']+SCALER_COL].to_html()))
    input('Next ...')
    clear_output()


In [ ]:
=========================================== TEST ==============================================

In [6]:
keys = None#df[unique_keys].sort_values(by=unique_keys).drop_duplicates()
max_option_len = 20
scaler = MinMaxScaler()
PartitionDate = [ d[-7:] for d in  os.listdir(PARQUET_PATH) if 'PartitionDate' in d]
options_qoute = {}
for i,partdate in enumerate(PartitionDate) :  
    df = pd.read_parquet(PARQUET_PATH,engine='pyarrow'
                                 , filters=[('PartitionDate', '=', partdate)]
                                )
    #add col options_id
    df['OPTIONS_ID'] = None
    df['P_VOLUME'] = df['P_VOLUME'].fillna(0)
    df['C_VOLUME'] = df['C_VOLUME'].fillna(0)

    ####################################################
    keys = df[UNIQUE_KEYS].sort_values(by=UNIQUE_KEYS).drop_duplicates()
    #loop each keys
    break_flag = False

    for j,v in keys.iterrows():
        
        df_filter=df[ (df['QUOTE_DATE'] == v['QUOTE_DATE']) & (df['SYMBOL'] == v['SYMBOL']) & (df['EXPIRE_DATE'] == v['EXPIRE_DATE']) ]
        qoute = "".join(v[ ['SYMBOL','EXPIRE_DATE'] ].apply(str).values)
        #add qoute
        if qoute not in [*options_qoute.keys()]:
            options_qoute[qoute] = {}
            options_qoute[qoute]['start_price'] = df_filter['UNDERLYING_LAST'].values[0]
            options_qoute[qoute]['strike'] = df_filter[ df_filter['INTRINSIC_VALUE'].abs().isin(df_filter['INTRINSIC_VALUE'].abs().sort_values()[:max_option_len]) ]['STRIKE'].values
            options_qoute[qoute]['exp'] = df_filter['EXPIRE_DATE'].values[0]
            #check diff UNDERLYING_LAST
            if df_filter['UNDERLYING_LAST'].values[0] != round(np.average(df_filter['UNDERLYING_LAST']),4):
                print('[ERROR] : set UNDERLYING_LAST ',qoute )
        #rm index max : max_option_len
        rm_strike_index = df_filter[ ~df_filter['STRIKE'].isin(options_qoute[qoute]['strike']) ].index
        if len( df_filter[ df_filter['STRIKE'].isin(options_qoute[qoute]['strike']) ] ) != 20 :
            break
        df = df.drop(rm_strike_index)
        df_filter = df_filter.drop(rm_strike_index)

        # Generate zero strike 
        # Generate a random float between 0.01 and 1
        random_number = random.uniform(0, 1)
        nom_rm_rows = [r  for r in [0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4] if random_number < r]
        strikeZero(df,v,len(nom_rm_rows) )
        #TOTAL_P_VOLUME,TOTAL_C_VOLUME,OPTIONS_ID
        df.loc[ (df['QUOTE_DATE'] == v['QUOTE_DATE']) 
        & (df['SYMBOL'] == v['SYMBOL']) 
        & (df['EXPIRE_DATE'] == v['EXPIRE_DATE']) 
        , ['TOTAL_VOLUME','OPTIONS_ID']
        ] = [
            df_filter['P_VOLUME'].sum() + df_filter['C_VOLUME'].sum(),
            hash_str(f'{i}{j}')
        ]
    if break_flag : 
        break
    #clear expire options_qoute
    for qi in list(options_qoute.keys()):
        if options_qoute[qi]['exp'] < df['QUOTE_DATE'].values[0]:
            options_qoute.pop(qi)

    ###################################################

    # scaler.partial_fit
    #scaler.partial_fit(df[SCALER_COL])
    print(f"[Processing] {partdate} {round(((i+1)/len(PartitionDate))*100,2)}%     ",end='\r')
    
    # save
    # if os.path.exists(PARQUET_STG_PATH):
    #   df.to_parquet(PARQUET_STG_PATH, engine='fastparquet', append=True, partition_cols=['PartitionDate'], index=False )
    # else:
    #   df.to_parquet(PARQUET_STG_PATH, engine='fastparquet' , partition_cols=['PartitionDate'], index=False  )
        
#joblib.dump(scaler, SCALER_PATH )

KeyboardInterrupt: 

20

,QUOTE_UNIXTIME,QUOTE_READTIME,QUOTE_DATE,QUOTE_TIME_HOURS,UNDERLYING_LAST,EXPIRE_DATE,EXPIRE_UNIX,DTE,C_DELTA,C_GAMMA,...,P_RHO,P_IV,P_VOLUME,STRIKE_DISTANCE,STRIKE_DISTANCE_PCT,SYMBOL,INTRINSIC_VALUE,PartitionDate,OPTIONS_ID,TOTAL_VOLUME
33407,1388782800,2014-01-03 16:00:00,2014-01-03,16.0,86.65,2014-01-17,1389992400,14.0,0.99940,0.00000,...,-0.00056,0.30138,29.0,9.7,0.111,QQQ,9.65,2014-01,None,NaN
33408,1388782800,2014-01-03 16:00:00,2014-01-03,16.0,86.65,2014-01-17,1389992400,14.0,1.00000,0.00000,...,-0.00054,0.28413,25.0,8.7,0.100,QQQ,8.65,2014-01,None,NaN
33409,1388782800,2014-01-03 16:00:00,2014-01-03,16.0,86.65,2014-01-17,1389992400,14.0,1.00000,0.00000,...,-0.00086,0.26222,2.0,7.7,0.088,QQQ,7.65,2014-01,None,NaN
33410,1388782800,2014-01-03 16:00:00,2014-01-03,16.0,86.65,2014-01-17,1389992400,14.0,1.00000,0.00000,...,-0.00147,0.23995,288.0,6.7,0.077,QQQ,6.65,2014-01,None,NaN
33411,1388782800,2014-01-03 16:00:00,2014-01-03,16.0,86.65,2014-01-17,1389992400,14.0,0.99959,0.00000,...,-0.00199,0.22078,305.0,5.7,0.065,QQQ,5.65,2014-01,None,NaN
33412,1388782800,2014-01-03 16:00:00,2014-01-03,16.0,86.65,2014-01-17,1389992400,14.0,0.95310,0.03318,...,-0.00247,0.19753,2212.0,4.7,0.054,QQQ,4.65,2014-01,None,NaN
33413,1388782800,2014-01-03 16:00:00,2014-01-03,16.0,86.65,2014-01-17,1389992400,14.0,0.92218,0.05274,...,-0.00353,0.18102,1955.0,3.7,0.042,QQQ,3.65,2014-01,None,NaN
33414,1388782800,2014-01-03 16:00:00,2014-01-03,16.0,86.65,2014-01-17,1389992400,14.0,0.88699,0.08108,...,-0.00586,0.16300,4523.0,2.7,0.031,QQQ,2.65,2014-01,None,NaN
33415,1388782800,2014-01-03 16:00:00,2014-01-03,16.0,86.65,2014-01-17,1389992400,14.0,0.78779,0.12657,...,-0.00882,0.15078,13417.0,1.7,0.019,QQQ,1.65,2014-01,None,NaN
33416,1388782800,2014-01-03 16:00:00,2014-01-03,16.0,86.65,2014-01-17,1389992400,14.0,0.64633,0.17211,...,-0.01288,0.14076,25064.0,0.7,0.007,QQQ,0.65,2014-01,None,NaN


In [12]:
df_filter['INTRINSIC_VALUE'].abs().isin(df_filter['INTRINSIC_VALUE'].abs().sort_values()[:max_option_len]

SyntaxError: incomplete input (3967381514.py, line 1)

In [9]:
df_filter

,QUOTE_UNIXTIME,QUOTE_READTIME,QUOTE_DATE,QUOTE_TIME_HOURS,UNDERLYING_LAST,EXPIRE_DATE,EXPIRE_UNIX,DTE,C_DELTA,C_GAMMA,...,P_RHO,P_IV,P_VOLUME,STRIKE_DISTANCE,STRIKE_DISTANCE_PCT,SYMBOL,INTRINSIC_VALUE,PartitionDate,OPTIONS_ID,TOTAL_VOLUME
33366,1388782800,2014-01-03 16:00:00,2014-01-03,16.0,86.65,2014-01-17,1389992400,14.0,0.99989,0.00000,...,-0.00020,1.47816,0.0,51.7,0.596,QQQ,51.65,2014-01,None,NaN
33367,1388782800,2014-01-03 16:00:00,2014-01-03,16.0,86.65,2014-01-17,1389992400,14.0,1.00000,0.00000,...,-0.00016,1.39140,0.0,49.7,0.573,QQQ,49.65,2014-01,None,NaN
33368,1388782800,2014-01-03 16:00:00,2014-01-03,16.0,86.65,2014-01-17,1389992400,14.0,0.99937,0.00000,...,-0.00012,1.34963,0.0,48.7,0.561,QQQ,48.65,2014-01,None,NaN
33369,1388782800,2014-01-03 16:00:00,2014-01-03,16.0,86.65,2014-01-17,1389992400,14.0,0.99960,0.00000,...,-0.00036,1.30943,0.0,47.7,0.550,QQQ,47.65,2014-01,None,NaN
33370,1388782800,2014-01-03 16:00:00,2014-01-03,16.0,86.65,2014-01-17,1389992400,14.0,1.00000,0.00000,...,-0.00015,1.27066,0.0,46.7,0.538,QQQ,46.65,2014-01,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33431,1388782800,2014-01-03 16:00:00,2014-01-03,16.0,86.65,2014-01-17,1389992400,14.0,0.00340,0.00210,...,0.00000,NaN,0.0,14.3,0.166,QQQ,-14.35,2014-01,None,NaN
33432,1388782800,2014-01-03 16:00:00,2014-01-03,16.0,86.65,2014-01-17,1389992400,14.0,0.00300,0.00197,...,0.00000,NaN,0.0,15.3,0.177,QQQ,-15.35,2014-01,None,NaN
33433,1388782800,2014-01-03 16:00:00,2014-01-03,16.0,86.65,2014-01-17,1389992400,14.0,0.00267,0.00175,...,0.00000,NaN,0.0,16.3,0.189,QQQ,-16.35,2014-01,None,NaN
33434,1388782800,2014-01-03 16:00:00,2014-01-03,16.0,86.65,2014-01-17,1389992400,14.0,0.00317,0.00167,...,0.00000,NaN,0.0,17.3,0.200,QQQ,-17.35,2014-01,None,NaN


In [ ]:
type(pd.util.hash_pandas_object(pd.Series([2,1])).sum() )

In [ ]:
pd.util.hash_pandas_object(pd.Series([2,0])).sum()

In [ ]:
pd.util.hash_pandas_object(pd.Series([0,2])).sum()

In [ ]:
import hashlib
a=hashlib.md5("12".encode('utf-8')).hexdigest() 

In [ ]:
def hash_str():
    hashlib.md5("01".encode('utf-8')).hexdigest() 

In [ ]:
a.hexdigest(20)